In [1]:
import json
import os
import re

In [3]:
#ściezka z danymi origin
input_folder_orig = r'D:\repo_git\ONS\data\chopin\orig'

#lista plikow origin
file_list_orig = ['1.json', '2.json', '3.json', '4.json', '5.json', '6.json','7.json', '8.json', '9.json', '10.json', '11.json', '12.json']

In [20]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ścieżki do plików
input_folder_orig = r'D:\repo_git\ONS\data\chopin\orig'
airports_csv = r'D:\repo_git\ONS\data\iata-icao.csv'

# Wczytanie danych lotnisk z pliku CSV
airports_df = pd.read_csv(airports_csv)

# Mapowanie krajów na kod ICAO lotnisk
country_to_icao = airports_df.set_index('icao')['country_code'].to_dict()
country = airports_df.set_index('icao')['region_name'].to_dict()

# Lista plików origin
file_list_orig = ['1.json', '2.json', '3.json', '4.json', '5.json', '6.json', '7.json', '8.json', '9.json', '10.json', '11.json', '12.json']

# Tworzenie listy do przechowywania wyników
all_flights = []

# Przetwarzanie plików z listy origin
for filename in file_list_orig:
    input_file_path = os.path.join(input_folder_orig, filename)

    with open(input_file_path, 'r', encoding='utf-8') as infile:
        for line in infile:
            try:
                data = json.loads(line)
                all_flights.append(data)
            except json.JSONDecodeError:
                print(f"Błąd dekodowania JSON w pliku {filename}, linia: {line}")

# Konwersja listy do DataFrame
df = pd.DataFrame(all_flights)

# Konwersja kolumny 'pitr' do formatu datetime
df['pitr'] = pd.to_datetime(df['pitr'])

# Dodanie kolumny 'month' na podstawie 'pitr'
df['month'] = df['pitr'].dt.month

# Zastąpienie kodów ICAO kodami krajów
df['dest_country'] = df['dest'].map(country_to_icao).fillna(df['dest'])




In [22]:
# Zliczenie ilości lotów z Polski do innych krajów w danym miesiącu
flights_from_poland_per_month = df[df['orig'] == 'EPWA'].groupby(['month', 'dest_country']).size()

# Przykładowe wyświetlenie dla miesiąca 1 i kraju PL (Polska)
month = 1
country = 'PL'
if (month, country) in flights_from_poland_per_month.index:
    count = flights_from_poland_per_month.loc[(month, country)]
    print(f"Liczba lotów z Polski do {country} w miesiącu {month}: {count}")
else:
    print(f"Brak lotów z Polski do {country} w miesiącu {month}")


Liczba lotów z Polski do PL w miesiącu 1: 727
